In [1]:
import os.path
from datetime import datetime, time

import pandas as pd
from lxml import etree

In [2]:
# First open up a file
input_filename = "./data/A321/Run01_A321_230426.xlsx"
trajectory_name = os.path.splitext(os.path.basename(input_filename))[0]
trajectory_original = pd.read_excel(input_filename, skiprows=[1])

# Show structure of data
trajectory_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Timestamp                       627 non-null    float64
 1   A/C LATITUDE                    627 non-null    float64
 2   A/C LONGITUDE                   627 non-null    float64
 3   ABSOLUTE HEIGHT ABOVE SEALEVEL  627 non-null    float64
 4   CALIBRATED AIRSPEED             627 non-null    float64
 5   GROUND SPEED                    627 non-null    float64
 6   TOTAL A/C VERTICAL SPEED        627 non-null    float64
 7   A/C ROLL ANGLE                  627 non-null    float64
 8   CORRECTED A/C HEADING           627 non-null    float64
 9   GROSS WEIGHT                    627 non-null    float64
 10  NET THRUST ENGINE 1             627 non-null    float64
 11  NET THRUST ENGINE 2             627 non-null    float64
 12  MACHNUMBER                      627 

In [3]:
# Rename columns
names = {
    "Timestamp": "timestamp",
    "A/C LATITUDE": "latitude",
    "A/C LONGITUDE": "longitude",
    "ABSOLUTE HEIGHT ABOVE SEALEVEL": "altitude",
    "CALIBRATED AIRSPEED": "calibrated_airspeed",
    "GROUND SPEED": "ground_speed",
    "TOTAL A/C VERTICAL SPEED": "vertical_speed",
    "TRUE TRACK ANGLE": "track"
}

In [4]:
trajectory = trajectory_original[names.keys()].rename(columns=names)
trajectory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            627 non-null    float64
 1   latitude             627 non-null    float64
 2   longitude            627 non-null    float64
 3   altitude             627 non-null    float64
 4   calibrated_airspeed  627 non-null    float64
 5   ground_speed         627 non-null    float64
 6   vertical_speed       627 non-null    float64
 7   track                627 non-null    float64
dtypes: float64(8)
memory usage: 39.3 KB


In [5]:
trajectory.describe()

,timestamp,latitude,longitude,altitude,calibrated_airspeed,ground_speed,vertical_speed,track
count,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000
mean,260385.800000,49.970811,8.593685,3012.197267,169.687003,179.973099,-27.582260,132.048039
std,90.571795,0.049301,0.084071,2737.912307,91.768272,99.583253,20.550927,72.396766
min,260229.300000,49.875470,8.525929,336.781100,0.008965,0.000000,-54.339850,-179.999800
25%,260307.550000,49.936980,8.526021,337.436200,122.138000,122.914250,-48.047505,118.076700
50%,260385.800000,49.962790,8.538898,2109.707000,204.518100,213.956000,-32.867810,144.432600
75%,260464.050000,50.025620,8.664182,5141.001500,248.976450,260.425900,-0.011413,179.316900
max,260542.300000,50.034170,8.779834,8971.250000,251.084300,283.530400,0.031867,179.970400


In [6]:
trajectory.head()

,timestamp,latitude,longitude,altitude,calibrated_airspeed,ground_speed,vertical_speed,track
0,260229.3,50.03417,8.525929,336.7844,0.103095,0.0,-0.000002,179.6355
1,260229.8,50.03417,8.525929,336.7844,0.103095,0.0,-0.000002,179.6355
2,260230.3,50.03417,8.525929,336.7844,0.103095,0.0,-0.000002,179.6355
3,260230.8,50.03417,8.525929,336.7844,0.103095,0.0,-0.000002,179.6355
4,260231.3,50.03417,8.525929,336.7844,0.103095,0.0,-0.000001,179.6355


In [7]:
# Calculate the elapsed time from the timestamps
trajectory["elapsed_seconds"] = trajectory["timestamp"] - trajectory["timestamp"].iloc[0]
trajectory["elapsed_seconds"].head()

0    0.0
1    0.5
2    1.0
3    1.5
4    2.0
Name: elapsed_seconds, dtype: float64

In [8]:
# Convert elapsed seconds to timedelta
trajectory["elapsed_seconds"] = pd.to_timedelta(trajectory["elapsed_seconds"], unit="seconds")
trajectory["elapsed_seconds"].head()

0          0 days 00:00:00
1   0 days 00:00:00.500000
2          0 days 00:00:01
3   0 days 00:00:01.500000
4          0 days 00:00:02
Name: elapsed_seconds, dtype: timedelta64[ns]

In [9]:
# Calculate an absolute time based on the start of today.
reference_datetime = datetime.combine(datetime.now().date(), time.min)
# Add the elapsed seconds to the reference date to obtain an absolute date for each datum
trajectory["datetime"] = reference_datetime + trajectory["elapsed_seconds"]
trajectory["datetime"].head()

0   2023-05-22 00:00:00.000
1   2023-05-22 00:00:00.500
2   2023-05-22 00:00:01.000
3   2023-05-22 00:00:01.500
4   2023-05-22 00:00:02.000
Name: datetime, dtype: datetime64[ns]

In [10]:
# Create a column with the formatted date as string
# Reference format 2022-02-28T01:00:37.950000+01:00
trajectory["datetime_str"] = trajectory["datetime"].dt.strftime("%Y-%m-%dT%H:%M:%S+00:00")
trajectory["datetime_str"].head()

0    2023-05-22T00:00:00+00:00
1    2023-05-22T00:00:00+00:00
2    2023-05-22T00:00:01+00:00
3    2023-05-22T00:00:01+00:00
4    2023-05-22T00:00:02+00:00
Name: datetime_str, dtype: object

In [43]:
# Start structuring the document
kml_namespace = "http://www.opengis.net/kml/2.2"
kml_ext_namespace = "http://www.google.com/kml/ext/2.2"

namespace_map = {
    "gx": "http://www.google.com/kml/ext/2.2",
    None: "http://www.opengis.net/kml/2.2"
}

# Create a new xml document
root = etree.Element("kml", nsmap=namespace_map)

# Add a root document
root_document = etree.SubElement(root, "Document")
root_document_name = etree.SubElement(root_document, "name")

root_document_name.text = trajectory_name

etree.tostring(root, xml_declaration=True, encoding="utf-8")

b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<kml xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns="http://www.opengis.net/kml/2.2"><Document><name>Run01_A321_230426</name></Document></kml>'

In [44]:
# Get the start and end point
start_point_coordinates = trajectory[["longitude", "latitude", "altitude"]].iloc[0].tolist()
end_point_coordinates = trajectory[["longitude", "latitude", "altitude"]].iloc[-1].tolist()

In [45]:
# add a LookAt element
look_at = etree.SubElement(root_document, "LookAt")
timespan_elem = etree.SubElement(look_at, f"{{{kml_ext_namespace}}}TimeSpan")
timespan_begin = etree.SubElement(timespan_elem, "begin")
timespan_begin.text = trajectory["datetime_str"].iloc[0]
timespan_end = etree.SubElement(timespan_elem, "end")
timespan_end.text = trajectory["datetime_str"].iloc[-1]

look_at_longitude = etree.SubElement(look_at, "longitude")
look_at_longitude.text = str(start_point_coordinates[0])
look_at_latitude = etree.SubElement(look_at, "latitude")
look_at_latitude.text = str(start_point_coordinates[1])
look_at_altitude = etree.SubElement(look_at, "altitude")
look_at_altitude.text = str(5000)
look_at_tilt = etree.SubElement(look_at, "tilt")
look_at_tilt.text = str(70)
look_at_altitude_mode = etree.SubElement(look_at, "altitudeMode")
look_at_altitude_mode.text = "absolute"

In [46]:
# Add a folder for the track animation
trajectory_4d_folder = etree.SubElement(root_document, "Folder")
trajectory_4d_folder_name = etree.SubElement(trajectory_4d_folder, "name")
trajectory_4d_folder_name.text = "4D Trajectory"

trajectory_4d_placemark = etree.SubElement(trajectory_4d_folder, "Placemark")
name_tag = etree.SubElement(trajectory_4d_placemark, "name")
name_tag.text = trajectory_name

track = etree.SubElement(trajectory_4d_placemark, f"{{{kml_ext_namespace}}}Track")

altitude_mode = etree.SubElement(track, "altitudeMode")
altitude_mode.text = "absolute"


In [47]:
# Add placemarks for start and end
start_placemark = etree.SubElement(trajectory_4d_folder, "Placemark")
start_placemark_name = etree.SubElement(start_placemark, "name")
start_placemark_name.text = "Start"
start_placemark_description = etree.SubElement(start_placemark, "description")
start_placemark_description.text = "Start of recording"
start_point = etree.SubElement(start_placemark, "Point")
start_point_coordinates_tag = etree.SubElement(start_point, "coordinates")
start_point_coordinates_tag.text = ",".join([str(c) for c in start_point_coordinates])
start_point_altitude_mode = etree.SubElement(start_point, "altitudeMode")
start_point_altitude_mode.text = "absolute"

end_placemark = etree.SubElement(trajectory_4d_folder, "Placemark")
end_placemark_name = etree.SubElement(end_placemark, "name")
end_placemark_name.text = "End"
end_placemark_description = etree.SubElement(end_placemark, "description")
end_placemark_description.text = "End of recording"
end_point = etree.SubElement(end_placemark, "Point")
end_point_coordinates_tag = etree.SubElement(end_point, "coordinates")
end_point_coordinates_tag.text = ",".join([str(c) for c in end_point_coordinates])
end_point_altitude_mode = etree.SubElement(end_point, "altitudeMode")
end_point_altitude_mode.text = "absolute"

In [48]:
# Get the 4d coordinates
coordinates_4d = list(trajectory[["longitude", "latitude", "altitude", "datetime_str"]].itertuples(index=False, name=None))

for longitude, latitude, altitude, t in coordinates_4d:
    when_tag = etree.SubElement(track, "when")
    when_tag.text = t
    coordinate_tag = etree.SubElement(track, f"{{{kml_ext_namespace}}}coord")
    coordinate_tag.text = f"{longitude} {latitude} {altitude}"

In [49]:
# Make 3D trajectory
trajectory_3d_folder = etree.SubElement(root_document, "Folder")
trajectory_3d_folder_name = etree.SubElement(trajectory_3d_folder, "name")
trajectory_3d_folder_name.text = "3D Trajectory"

trajectory_3d_placemark = etree.SubElement(trajectory_3d_folder, "Placemark")
trajectory_3d_placemark_name = etree.SubElement(trajectory_3d_placemark, "name")
trajectory_3d_folder_name.text = "3D Trajectory"

line_string = etree.SubElement(trajectory_3d_placemark, "LineString")
extrude = etree.SubElement(line_string, "extrude")
extrude.text = str(1)

altitude_mode = etree.SubElement(line_string, "altitudeMode")
altitude_mode.text = "absolute"

coordinates_tag = etree.SubElement(line_string, "coordinates")
coordinates_elems = [f"{latitude},{longitude},{altitude}" for latitude, longitude, altitude, _ in coordinates_4d]
coordinates_tag.text = " ".join(coordinates_elems)


In [50]:
# Save xml to disk

output_dir = os.path.dirname(input_filename)
output_file = os.path.join(output_dir, f"{trajectory_name}.kml")

tree = etree.ElementTree(root)
tree.write(output_file, xml_declaration=True, encoding="utf-8", pretty_print=True)

root.nsmap

{'gx': 'http://www.google.com/kml/ext/2.2',
 None: 'http://www.opengis.net/kml/2.2'}